# This code cleans the Rideshare Trip data 

It removes duplicates, trips under 2 miles, and trips over 1 day. 
It then exports the remaining data to a csv. Appropriate for 1 month of data at a time. 

In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import re

In [2]:
rs = pd.read_csv('data/Chicago/Rideshare_M1.csv') #reads rideshare trip information as rs

In [3]:
rs

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,0023918a32baa84c59a60ec6acd38ef8d8876a55,01/01/2022 12:00:00 AM,01/01/2022 12:45:00 AM,2123,27.8,NaN,NaN,8.0,NaN,37.5,...,7.91,45.41,False,1,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),NaN,NaN,NaN
1,0031f13d0144700e8132ec4ced74bf91b604bafd,01/01/2022 12:00:00 AM,01/01/2022 12:00:00 AM,308,1.4,1.703107e+10,1.703108e+10,7.0,8.0,5.0,...,3.75,8.75,False,1,41.922083,-87.634156,POINT (-87.6341560931 41.922082541),41.907413,-87.640902,POINT (-87.6409015248 41.9074128162)
2,003b7701b5658afe3415169d091130f15c2d8434,01/01/2022 12:00:00 AM,01/01/2022 12:15:00 AM,830,3.6,1.703108e+10,1.703122e+10,8.0,22.0,10.0,...,1.02,11.02,False,1,41.907492,-87.635760,POINT (-87.6357600901 41.9074919303),41.920802,-87.694532,POINT (-87.6945323419 41.9208017043)
3,003c710f7464bf348349ec4436fea275b97300bd,01/01/2022 12:00:00 AM,01/01/2022 12:15:00 AM,862,4.7,1.703122e+10,1.703108e+10,22.0,8.0,10.0,...,3.74,13.74,False,1,41.915910,-87.683826,POINT (-87.6838259399 41.9159097884),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595)
4,0066f1154bb7dc467b2d8ce59e8def475a428557,01/01/2022 12:00:00 AM,01/01/2022 12:00:00 AM,471,0.7,1.703108e+10,1.703108e+10,8.0,8.0,2.5,...,2.36,4.86,False,1,41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4704324,fee2f42777fe6d5c0d01273a1801995746e6ae7b,01/31/2022 11:45:00 PM,01/31/2022 11:45:00 PM,262,1.2,1.703108e+10,1.703183e+10,8.0,28.0,12.5,...,2.36,14.86,False,1,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201)
4704325,ff31acc0f2a44c820d498777aa70c5e95adff275,01/31/2022 11:45:00 PM,02/01/2022 12:00:00 AM,596,1.0,NaN,NaN,49.0,49.0,5.0,...,2.36,7.36,False,1,41.706588,-87.623367,POINT (-87.6233665115 41.7065878819),41.706588,-87.623367,POINT (-87.6233665115 41.7065878819)
4704326,ffb1bdcdf9e54d77f14f39549981729e7c76294b,01/31/2022 11:45:00 PM,02/01/2022 12:00:00 AM,524,3.0,NaN,NaN,7.0,8.0,7.5,...,1.02,10.52,False,1,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
4704327,ffc75f5072b9f7ee8b5c940abbbc23a1dc8cf65e,01/31/2022 11:45:00 PM,02/01/2022 12:00:00 AM,772,5.8,NaN,NaN,30.0,32.0,10.0,...,4.16,14.16,False,1,41.839087,-87.714004,POINT (-87.714003807 41.8390869059),41.878866,-87.625192,POINT (-87.6251921424 41.8788655841)


In [4]:
rs.shape

(4704329, 21)

In [5]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4704329 entries, 0 to 4704328
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Trip Start Timestamp        object 
 2   Trip End Timestamp          object 
 3   Trip Seconds                int64  
 4   Trip Miles                  float64
 5   Pickup Census Tract         float64
 6   Dropoff Census Tract        float64
 7   Pickup Community Area       float64
 8   Dropoff Community Area      float64
 9   Fare                        float64
 10  Tip                         float64
 11  Additional Charges          float64
 12  Trip Total                  float64
 13  Shared Trip Authorized      bool   
 14  Trips Pooled                int64  
 15  Pickup Centroid Latitude    float64
 16  Pickup Centroid Longitude   float64
 17  Pickup Centroid Location    object 
 18  Dropoff Centroid Latitude   float64
 19  Dropoff Centroid Long

In [6]:
rs.describe()

,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tip,Additional Charges,Trip Total,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude
count,4.704329e+06,4.704143e+06,2.479952e+06,2.466361e+06,4.361344e+06,4.321158e+06,4.703313e+06,4.703313e+06,4.703313e+06,4.703313e+06,4.704329e+06,4.368325e+06,4.368325e+06,4.328462e+06,4.328462e+06
mean,9.326886e+02,6.216611e+00,1.703137e+10,1.703137e+10,2.680213e+01,2.689497e+01,1.489196e+01,8.644088e-01,3.724454e+00,1.948082e+01,1.000010e+00,4.188939e+01,-8.767048e+01,4.188929e+01,-8.767027e+01
std,6.455552e+02,7.379275e+00,3.399275e+05,3.397413e+05,2.148010e+01,2.146048e+01,1.179623e+01,2.296140e+00,3.586732e+00,1.448545e+01,3.324689e-03,6.864211e-02,6.601218e-02,6.819730e-02,6.681474e-02
min,2.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.165022e+01,-8.791362e+01,4.165022e+01,-8.791362e+01
25%,4.820000e+02,1.800000e+00,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,7.500000e+00,0.000000e+00,2.360000e+00,1.089000e+01,1.000000e+00,4.187061e+01,-8.768932e+01,4.187102e+01,-8.768752e+01
50%,7.750000e+02,3.600000e+00,1.703124e+10,1.703124e+10,2.400000e+01,2.400000e+01,1.250000e+01,0.000000e+00,2.840000e+00,1.546000e+01,1.000000e+00,4.189831e+01,-8.765591e+01,4.189503e+01,-8.765588e+01
75%,1.223000e+03,7.800000e+00,1.703183e+10,1.703183e+10,3.300000e+01,3.300000e+01,1.750000e+01,0.000000e+00,4.360000e+00,2.283000e+01,1.000000e+00,4.193609e+01,-8.763186e+01,4.193476e+01,-8.763186e+01
max,7.755600e+04,8.986000e+02,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,1.227500e+03,2.000000e+02,3.028600e+02,1.255220e+03,3.000000e+00,4.202122e+01,-8.753071e+01,4.202122e+01,-8.752995e+01


In [7]:
rs.loc[(rs.duplicated(subset=['Trip ID'], keep='first') == True)] #checking for duplicate ids

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,...,Additional Charges,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location


In [12]:
#rs_clean = rs.loc[(rs_clean.duplicated(subset=['Trip ID'], keep='first') == False)] 
#removes duplicate values of trip id

In [8]:
#filters out trips shorter than 2 miles and longer than 1 day
rs_clean = rs.loc[(rs['Trip Miles'] >= 2) & (rs['Trip Seconds'] <= 86400)] 
rs_clean.shape

(3383612, 21)

In [12]:
rs_clean.to_csv('data/Chicago/cleaned/rs_clean_M1.csv', index=False)
#exports clean dataset to csv

In [9]:
rs_clean['Trip Start Timestamp'].min()

'01/01/2022 01:00:00 AM'

In [10]:
rs_clean['Trip End Timestamp'].max()

'02/01/2022 12:45:00 AM'

In [13]:
#exports sample csv
rs10000 = rs_clean.head(10000)
rs10000.to_csv('data/Chicago/cleaned/rs10000.csv', index=False)